# Modelling 1

### Library and data imports

In [1]:
from importlib import reload
import pandas as pd
import numpy as np
from spektral.data import Dataset, Graph
import dataset
reload(dataset)
import pickle
import os

from spektral.data import BatchLoader
from spektral.layers import GATConv, GCNConv, GlobalAvgPool, GlobalMaxPool, GlobalSumPool, GlobalAttentionPool
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import binary_accuracy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.config import list_physical_devices
from tensorflow.python.client import device_lib
import random

In [2]:
# Load combined data
df_raw = pd.read_csv('../data/combined.csv')
df = df_raw.copy()

# Load hero feature data
df_features = pd.read_csv('../data/features.csv')
df_features = df_features.set_index('hero_id')

# Load standard filter
df_filters = pd.read_csv('../models/filters.csv')

In [3]:
# Load graph dataset 50000 matches at a time
dir = '../data/graphs_v1_scaled/'
count = 0
total = len(df)
step = 50000

for i in range(0,int(np.ceil(total/step))):
    start = i*step
    end = start+step-1 if (start+step)<total else total-1
    path = dir+f'graphs_v1_scaled_{start}-{end}.pkl'
    print(path)
    file = open(path,'rb')
    if i==0:
        graphs = pickle.load(file)
    else:
        graphs = graphs + pickle.load(file)

../data/graphs_v1_scaled/graphs_v1_scaled_0-49999.pkl
../data/graphs_v1_scaled/graphs_v1_scaled_50000-99999.pkl
../data/graphs_v1_scaled/graphs_v1_scaled_100000-149999.pkl
../data/graphs_v1_scaled/graphs_v1_scaled_150000-199999.pkl
../data/graphs_v1_scaled/graphs_v1_scaled_200000-249999.pkl
../data/graphs_v1_scaled/graphs_v1_scaled_250000-299999.pkl
../data/graphs_v1_scaled/graphs_v1_scaled_300000-349999.pkl
../data/graphs_v1_scaled/graphs_v1_scaled_350000-399999.pkl
../data/graphs_v1_scaled/graphs_v1_scaled_400000-449999.pkl
../data/graphs_v1_scaled/graphs_v1_scaled_450000-499999.pkl
../data/graphs_v1_scaled/graphs_v1_scaled_500000-549999.pkl
../data/graphs_v1_scaled/graphs_v1_scaled_550000-599999.pkl
../data/graphs_v1_scaled/graphs_v1_scaled_600000-649999.pkl
../data/graphs_v1_scaled/graphs_v1_scaled_650000-699999.pkl
../data/graphs_v1_scaled/graphs_v1_scaled_700000-749999.pkl
../data/graphs_v1_scaled/graphs_v1_scaled_750000-799999.pkl
../data/graphs_v1_scaled/graphs_v1_scaled_800000

In [4]:
def get_filt_idx(filt):
    '''Returns indices of desired matches given a boolean array filter e.g. True, False, True returns [0,2]'''
    # DotaV1 data handling (two graphs for every match: 0-49999 radiant, 0-49999 dire, 50000-99999 radiant, etc.)
    step = 50000
    filt_vals = []
    for i in range(0,int(np.ceil(len(filt)/step))):
        start = i*step
        end = start+step
        # Add filters for match range twice, as matches repeated every 50000
        filt_vals = np.append(filt_vals, filt[start:end])
        filt_vals = np.append(filt_vals, filt[start:end])

    # Get indices of True values in filters
    filt_idx = [i for i, x in enumerate(filt_vals) if x]
    return filt_idx

### Standard Filtering

In [5]:
# Scaling - data now pre-scaled
# # MinMax Scaler model to normalise features from 0-1
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# scaler.fit(df_features.iloc[:,3:].to_numpy())

# # Loop through each graph and scale feature matrix and drop attack_backswing feature
# print('Scaling graph dataset feature matrices:')
# for i in range(0,len(graphs_filt)):
#     if(i%100000==0):
#         print(i)
#     graphs_filt[i].x = scaler.transform(graphs_filt[i].x) # scale feature matrix
#     graphs_filt[i].x = np.delete(graphs_filt[i].x, 14, 1) # remove attack_backswing as a feature

In [6]:
# Filter graph dataset
filt = df_filters['filt_std'].values
filt_idx = get_filt_idx(filt)
graphs_filt = graphs[filt_idx]
print('Standard filtering complete')

Standard filtering complete


In [7]:
# Remove attack_backswing feature
for i in range(0,len(graphs_filt)):
    # if(i%100000==0):
    graphs_filt[i].x = graphs_filt[i].x[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,15,16,17,18,19]] # remove attack_backswing as a feature
print('Attack backswing feature removed')

Attack backswing feature removed


In [8]:
# Train/valid/test split
d = graphs_filt # Graph data

np.random.seed(10)
idxs = np.random.permutation(len(d))
split_va, split_te = int(0.64 * len(d)), int(0.8 * len(d)) #64% training, 16% validation, 20% test
idx_tr, idx_va, idx_te = np.split(idxs, [split_va, split_te])
data_tr = d[idx_tr]
data_va = d[idx_va]
data_te = d[idx_te]

print(f'Training data: {np.round(len(data_tr)/len(graphs_filt),2)*100}%')
print(f'Validation data: {np.round(len(data_va)/len(graphs_filt),2)*100}%')
print(f'Test data: {np.round(len(data_te)/len(graphs_filt),2)*100}%')

Training data: 64.0%
Validation data: 16.0%
Test data: 20.0%


In [1]:
# Confirm GPUs are being identified (requires tf environment, not the pipenv dotaprediction)
# print("Num GPUs Available: ", list_physical_devices('GPU'))
# print(device_lib.list_local_devices())

### Model 1.0 - Baseline

In [99]:
# Configuration
learning_rate = 0.001  # Learning rate
epochs = 50  # Number of training epochs
es_patience = 10  # Patience for early stopping
batch_size = 256  # Batch size

# Data loaders
loader_tr = BatchLoader(data_tr, batch_size=batch_size, epochs=epochs)
loader_va = BatchLoader(data_va, batch_size=batch_size)
loader_te = BatchLoader(data_te, batch_size=batch_size)

# Build model
class Net_1_0(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(19, activation='relu')
        self.flatten = Flatten()
        self.dense = Dense(d.n_labels, activation="sigmoid")

    def call(self, inputs):
        x, a = inputs
        # x.shape is [n_batches, 5, n_ features]
        # a.shape is [n_batches, 5, 5]

        x = self.conv1([x, a])
        # x.shape is [n_batches, 5, channels] where I have set channels = features (20)

        x = self.flatten(x)
        # x.shape is [n_batches, 5*channels]

        x = self.dense(x)
        # output.shape is [n_batches, 1]
        
        return x

model_1_0 = Net_1_0()
optimizer = Adam(learning_rate=learning_rate)
loss_fn = BinaryCrossentropy()
model_1_0.compile(optimizer=optimizer, loss=loss_fn, metrics=['binary_accuracy'])

fit_log_1_0 = model_1_0.fit(loader_tr.load(), steps_per_epoch=loader_tr.steps_per_epoch, epochs=epochs, validation_data=loader_va.load(), validation_steps=loader_va.steps_per_epoch)

c:\Users\nick_\.virtualenvs\dotaprediction-eZ2WDOkz\lib\site-packages\spektral\data\utils.py:221: UserWarning:

you are shuffling a 'DotaV1' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.



Epoch 1/50
25391/25391 [==============================] - 225s 9ms/step - loss: 0.6922 - binary_accuracy: 0.5185 - val_loss: 0.6915 - val_binary_accuracy: 0.5228
Epoch 2/50
25391/25391 [==============================] - 133s 5ms/step - loss: 0.6914 - binary_accuracy: 0.5241 - val_loss: 0.6914 - val_binary_accuracy: 0.5238
Epoch 3/50
25391/25391 [==============================] - 135s 5ms/step - loss: 0.6912 - binary_accuracy: 0.5252 - val_loss: 0.6918 - val_binary_accuracy: 0.5228
Epoch 4/50
25391/25391 [==============================] - 136s 5ms/step - loss: 0.6912 - binary_accuracy: 0.5256 - val_loss: 0.6918 - val_binary_accuracy: 0.5222
Epoch 5/50
25391/25391 [==============================] - 149s 6ms/step - loss: 0.6911 - binary_accuracy: 0.5260 - val_loss: 0.6910 - val_binary_accuracy: 0.5266
Epoch 6/50
25391/25391 [==============================] - 131s 5ms/step - loss: 0.6911 - binary_accuracy: 0.5264 - val_loss: 0.6912 - val_binary_accuracy: 0.5252
Epoch 7/50
25391/25391 [====

In [100]:
import plotly.graph_objects as go

epochs = list(range(1,len(fit_log_1_0.history['binary_accuracy'])+1))
training_accuracy = fit_log_1_0.history['binary_accuracy']
validation_accuracy = fit_log_1_0.history['val_binary_accuracy']

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=epochs,
    y=training_accuracy,
    mode='lines',
    name='Training'))
fig.add_trace(go.Scatter(
    x=epochs,
    y=validation_accuracy,
    mode='lines',
    name='Validation'))
fig.update_layout(
    template='none',
    xaxis_title='Epochs',
    yaxis_title="Accuracy")
fig.update_xaxes(tick0=0, dtick=1)
# fig.update_yaxes(range=[0.5,1])
fig.show()
fig.write_image(f'../images/model_1_0_accuracy.png', scale=5)
pd.DataFrame({'epoch': epochs, 'training_accuracy':training_accuracy, 'validation_accuracy':validation_accuracy}).to_csv('../images/model_1_0_accuracy.csv', index=False)

In [109]:
filehandler = open(f'../models/fit_log_1_0.pkl','wb')
pickle.dump(fit_log_1_0, filehandler)
filehandler = open(f'../models/model_1_0.pkl','wb')
pickle.dump(model_1_0, filehandler)

INFO:tensorflow:Assets written to: ram://639f241c-8626-491f-b1e7-68ee9353ea29/assets
INFO:tensorflow:Assets written to: ram://d5d9e920-b04a-42b1-9c2f-a6666cc80e66/assets


### Model 1.1 - GATConv

In [101]:
# Configuration
learning_rate = 0.001  # Learning rate
epochs = 50  # Number of training epochs
es_patience = 10  # Patience for early stopping
batch_size = 256  # Batch size

# Data loaders
loader_tr = BatchLoader(data_tr, batch_size=batch_size, epochs=epochs)
loader_va = BatchLoader(data_va, batch_size=batch_size)
loader_te = BatchLoader(data_te, batch_size=batch_size)

# Build model
class Net_1_1(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = GATConv(19, activation='relu')
        self.flatten = Flatten()
        self.dense = Dense(d.n_labels, activation="sigmoid")

    def call(self, inputs):
        x, a = inputs
        # x.shape is [n_batches, 5, n_ features]
        # a.shape is [n_batches, 5, 5]

        x = self.conv1([x, a])
        # x.shape is [n_batches, 5, channels] where I have set channels = features (20)

        x =self.flatten(x)
        # x.shape is [n_batches, 5*channels]

        x = self.dense(x)
        # output.shape is [n_batches, 1]
        
        return x

model_1_1 = Net_1_1()
optimizer = Adam(learning_rate=learning_rate)
loss_fn = BinaryCrossentropy()
model_1_1.compile(optimizer=optimizer, loss=loss_fn, metrics=['binary_accuracy'])

fit_log_1_1 = model_1_1.fit(loader_tr.load(), steps_per_epoch=loader_tr.steps_per_epoch, epochs=epochs, validation_data=loader_va.load(), validation_steps=loader_va.steps_per_epoch)

Epoch 1/50
25391/25391 [==============================] - 220s 9ms/step - loss: 0.6913 - binary_accuracy: 0.5259 - val_loss: 0.6904 - val_binary_accuracy: 0.5317
Epoch 2/50
25391/25391 [==============================] - 213s 8ms/step - loss: 0.6908 - binary_accuracy: 0.5300 - val_loss: 0.6902 - val_binary_accuracy: 0.5332
Epoch 3/50
25391/25391 [==============================] - 209s 8ms/step - loss: 0.6905 - binary_accuracy: 0.5312 - val_loss: 0.6903 - val_binary_accuracy: 0.5316
Epoch 4/50
25391/25391 [==============================] - 210s 8ms/step - loss: 0.6903 - binary_accuracy: 0.5322 - val_loss: 0.6898 - val_binary_accuracy: 0.5336
Epoch 5/50
25391/25391 [==============================] - 209s 8ms/step - loss: 0.6901 - binary_accuracy: 0.5327 - val_loss: 0.6904 - val_binary_accuracy: 0.5304
Epoch 6/50
25391/25391 [==============================] - 208s 8ms/step - loss: 0.6900 - binary_accuracy: 0.5333 - val_loss: 0.6900 - val_binary_accuracy: 0.5326
Epoch 7/50
25391/25391 [====

In [102]:
import plotly.graph_objects as go

epochs = list(range(1,len(fit_log_1_1.history['binary_accuracy'])+1))
training_accuracy = fit_log_1_1.history['binary_accuracy']
validation_accuracy = fit_log_1_1.history['val_binary_accuracy']

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=epochs,
    y=training_accuracy,
    mode='lines',
    name='Training'))
fig.add_trace(go.Scatter(
    x=epochs,
    y=validation_accuracy,
    mode='lines',
    name='Validation'))
fig.update_layout(
    template='none',
    xaxis_title='Epochs',
    yaxis_title="Accuracy")
fig.update_xaxes(tick0=0, dtick=1)
# fig.update_yaxes(range=[0.5,1])
fig.show()
fig.write_image(f'../images/model_1_1_accuracy.png', scale=5)
pd.DataFrame({'epoch': epochs, 'training_accuracy':training_accuracy, 'validation_accuracy':validation_accuracy}).to_csv('../images/model_1_1_accuracy.csv', index=False)

In [110]:
filehandler = open(f'../models/fit_log_1_1.pkl','wb')
pickle.dump(fit_log_1_1, filehandler)
filehandler = open(f'../models/model_1_1.pkl','wb')
pickle.dump(model_1_1, filehandler)

INFO:tensorflow:Assets written to: ram://956608be-fc57-4333-aef7-11622328db6f/assets


INFO:tensorflow:Assets written to: ram://956608be-fc57-4333-aef7-11622328db6f/assets


INFO:tensorflow:Assets written to: ram://597eca29-d923-4855-b598-ef1201bbb3d0/assets


INFO:tensorflow:Assets written to: ram://597eca29-d923-4855-b598-ef1201bbb3d0/assets


### Model 1.2 Average Pooling

In [103]:
# Configuration
learning_rate = 0.001  # Learning rate
epochs = 50  # Number of training epochs
es_patience = 10  # Patience for early stopping
batch_size = 256  # Batch size

# Data loaders
loader_tr = BatchLoader(data_tr, batch_size=batch_size, epochs=epochs)
loader_va = BatchLoader(data_va, batch_size=batch_size)
loader_te = BatchLoader(data_te, batch_size=batch_size)

# Build model
class Net_1_2(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(19, activation='relu')
        self.pool1 = GlobalAvgPool()
        self.flatten = Flatten()
        self.dense = Dense(d.n_labels, activation="sigmoid")

    def call(self, inputs):
        x, a = inputs
        # x.shape is [n_batches, 5, n_ features]
        # a.shape is [n_batches, 5, 5]

        x = self.conv1([x, a])
        # x.shape is [n_batches, 5, channels] where I have set channels = features (20)

        x = self.pool1(x)
        # x.shape is [n_batches, channels]

        x = self.dense(x)
        # output.shape is [n_batches, 1]
        
        return x

model_1_2 = Net_1_2()
optimizer = Adam(learning_rate=learning_rate)
loss_fn = BinaryCrossentropy()
model_1_2.compile(optimizer=optimizer, loss=loss_fn, metrics=['binary_accuracy'])

fit_log_1_2 = model_1_2.fit(loader_tr.load(), steps_per_epoch=loader_tr.steps_per_epoch, epochs=epochs, validation_data=loader_va.load(), validation_steps=loader_va.steps_per_epoch)

Epoch 1/50
25391/25391 [==============================] - 138s 5ms/step - loss: 0.6931 - binary_accuracy: 0.5156 - val_loss: 0.6914 - val_binary_accuracy: 0.5236
Epoch 2/50
25391/25391 [==============================] - 132s 5ms/step - loss: 0.6918 - binary_accuracy: 0.5214 - val_loss: 0.6915 - val_binary_accuracy: 0.5231
Epoch 3/50
25391/25391 [==============================] - 132s 5ms/step - loss: 0.6914 - binary_accuracy: 0.5237 - val_loss: 0.6911 - val_binary_accuracy: 0.5255
Epoch 4/50
25391/25391 [==============================] - 132s 5ms/step - loss: 0.6912 - binary_accuracy: 0.5249 - val_loss: 0.6909 - val_binary_accuracy: 0.5273
Epoch 5/50
25391/25391 [==============================] - 132s 5ms/step - loss: 0.6911 - binary_accuracy: 0.5260 - val_loss: 0.6908 - val_binary_accuracy: 0.5276
Epoch 6/50
25391/25391 [==============================] - 132s 5ms/step - loss: 0.6910 - binary_accuracy: 0.5264 - val_loss: 0.6910 - val_binary_accuracy: 0.5265
Epoch 7/50
25391/25391 [====

In [104]:
import plotly.graph_objects as go

epochs = list(range(1,len(fit_log_1_2.history['binary_accuracy'])+1))
training_accuracy = fit_log_1_2.history['binary_accuracy']
validation_accuracy = fit_log_1_2.history['val_binary_accuracy']

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=epochs,
    y=training_accuracy,
    mode='lines',
    name='Training'))
fig.add_trace(go.Scatter(
    x=epochs,
    y=validation_accuracy,
    mode='lines',
    name='Validation'))
fig.update_layout(
    template='none',
    xaxis_title='Epochs',
    yaxis_title="Accuracy")
fig.update_xaxes(tick0=0, dtick=1)
# fig.update_yaxes(range=[0.5,1])
fig.show()
fig.write_image(f'../images/model_1_2_accuracy.png', scale=5)
pd.DataFrame({'epoch': epochs, 'training_accuracy':training_accuracy, 'validation_accuracy':validation_accuracy}).to_csv('../images/model_1_2_accuracy.csv', index=False)

In [111]:
filehandler = open(f'../models/fit_log_1_2.pkl','wb')
pickle.dump(fit_log_1_2, filehandler)
filehandler = open(f'../models/model_1_2.pkl','wb')
pickle.dump(model_1_2, filehandler)

INFO:tensorflow:Assets written to: ram://29ccb6c4-b7a8-4122-a360-a938ff6d948a/assets


INFO:tensorflow:Assets written to: ram://29ccb6c4-b7a8-4122-a360-a938ff6d948a/assets
c:\Users\nick_\.virtualenvs\dotaprediction-eZ2WDOkz\lib\site-packages\keras\saving\saved_model\layer_serialization.py:112: CustomMaskWarning:

Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.



INFO:tensorflow:Assets written to: ram://59ee2602-97ad-4795-9064-2876f2390e32/assets


INFO:tensorflow:Assets written to: ram://59ee2602-97ad-4795-9064-2876f2390e32/assets
c:\Users\nick_\.virtualenvs\dotaprediction-eZ2WDOkz\lib\site-packages\keras\saving\saved_model\layer_serialization.py:112: CustomMaskWarning:

Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.



### Model 1.3 - Max Pooling

In [105]:
# Configuration
learning_rate = 0.001  # Learning rate
epochs = 50  # Number of training epochs
es_patience = 10  # Patience for early stopping
batch_size = 256  # Batch size

# Data loaders
loader_tr = BatchLoader(data_tr, batch_size=batch_size, epochs=epochs)
loader_va = BatchLoader(data_va, batch_size=batch_size)
loader_te = BatchLoader(data_te, batch_size=batch_size)

# Build model
class Net_1_3(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(19, activation='relu')
        self.pool1 = GlobalMaxPool()
        self.dense = Dense(d.n_labels, activation="sigmoid")

    def call(self, inputs):
        x, a = inputs
        # x.shape is [n_batches, 5, n_ features]
        # a.shape is [n_batches, 5, 5]

        x = self.conv1([x, a])
        # x.shape is [n_batches, 5, channels] where I have set channels = features (20)

        x = self.pool1(x)
        # x.shape is [n_batches, channels]

        x = self.dense(x)
        # output.shape is [n_batches, 1]
        
        return x

model_1_3 = Net_1_3()
optimizer = Adam(learning_rate=learning_rate)
loss_fn = BinaryCrossentropy()
model_1_3.compile(optimizer=optimizer, loss=loss_fn, metrics=['binary_accuracy'])

fit_log_1_3 = model_1_3.fit(loader_tr.load(), steps_per_epoch=loader_tr.steps_per_epoch, epochs=epochs, validation_data=loader_va.load(), validation_steps=loader_va.steps_per_epoch)

Epoch 1/50
25391/25391 [==============================] - 163s 6ms/step - loss: 0.6929 - binary_accuracy: 0.5158 - val_loss: 0.6918 - val_binary_accuracy: 0.5203
Epoch 2/50
25391/25391 [==============================] - 155s 6ms/step - loss: 0.6917 - binary_accuracy: 0.5218 - val_loss: 0.6912 - val_binary_accuracy: 0.5251
Epoch 3/50
25391/25391 [==============================] - 159s 6ms/step - loss: 0.6913 - binary_accuracy: 0.5244 - val_loss: 0.6912 - val_binary_accuracy: 0.5258
Epoch 4/50
25391/25391 [==============================] - 153s 6ms/step - loss: 0.6911 - binary_accuracy: 0.5256 - val_loss: 0.6911 - val_binary_accuracy: 0.5252
Epoch 5/50
25391/25391 [==============================] - 157s 6ms/step - loss: 0.6910 - binary_accuracy: 0.5263 - val_loss: 0.6912 - val_binary_accuracy: 0.5244
Epoch 6/50
25391/25391 [==============================] - 157s 6ms/step - loss: 0.6909 - binary_accuracy: 0.5268 - val_loss: 0.6913 - val_binary_accuracy: 0.5248
Epoch 7/50
25391/25391 [====

In [106]:
import plotly.graph_objects as go

epochs = list(range(1,len(fit_log_1_3.history['binary_accuracy'])+1))
training_accuracy = fit_log_1_3.history['binary_accuracy']
validation_accuracy = fit_log_1_3.history['val_binary_accuracy']

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=epochs,
    y=training_accuracy,
    mode='lines',
    name='Training'))
fig.add_trace(go.Scatter(
    x=epochs,
    y=validation_accuracy,
    mode='lines',
    name='Validation'))
fig.update_layout(
    template='none',
    xaxis_title='Epochs',
    yaxis_title="Accuracy")
fig.update_xaxes(tick0=0, dtick=1)
# fig.update_yaxes(range=[0.5,1])
fig.show()
fig.write_image(f'../images/model_1_3_accuracy.png', scale=5)
pd.DataFrame({'epoch': epochs, 'training_accuracy':training_accuracy, 'validation_accuracy':validation_accuracy}).to_csv('../images/model_1_3_accuracy.csv', index=False)

In [112]:
filehandler = open(f'../models/fit_log_1_3.pkl','wb')
pickle.dump(fit_log_1_3, filehandler)
filehandler = open(f'../models/model_1_3.pkl','wb')
pickle.dump(model_1_3, filehandler)

INFO:tensorflow:Assets written to: ram://e9df102b-d0e4-4da4-8204-d62605a8d8ac/assets


INFO:tensorflow:Assets written to: ram://e9df102b-d0e4-4da4-8204-d62605a8d8ac/assets
c:\Users\nick_\.virtualenvs\dotaprediction-eZ2WDOkz\lib\site-packages\keras\saving\saved_model\layer_serialization.py:112: CustomMaskWarning:

Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.



INFO:tensorflow:Assets written to: ram://76b5acdc-4d22-4a01-9499-96668dc37710/assets


INFO:tensorflow:Assets written to: ram://76b5acdc-4d22-4a01-9499-96668dc37710/assets
c:\Users\nick_\.virtualenvs\dotaprediction-eZ2WDOkz\lib\site-packages\keras\saving\saved_model\layer_serialization.py:112: CustomMaskWarning:

Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.



### Model 1.4 - Sum Pooling

In [10]:
# Configuration
learning_rate = 0.001  # Learning rate
epochs = 50  # Number of training epochs
es_patience = 10  # Patience for early stopping
batch_size = 256  # Batch size

# Data loaders
loader_tr = BatchLoader(data_tr, batch_size=batch_size, epochs=epochs)
loader_va = BatchLoader(data_va, batch_size=batch_size)
loader_te = BatchLoader(data_te, batch_size=batch_size)

# Build model
class Net_1_4(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(19, activation='relu')
        self.pool1 = GlobalSumPool()
        self.dense = Dense(d.n_labels, activation="sigmoid")

    def call(self, inputs):
        x, a = inputs
        # x.shape is [n_batches, 5, n_ features]
        # a.shape is [n_batches, 5, 5]

        x = self.conv1([x, a])
        # x.shape is [n_batches, 5, channels] where I have set channels = features (20)

        x = self.pool1(x)
        # x.shape is [n_batches, channels]

        x = self.dense(x)
        # output.shape is [n_batches, 1]
        
        return x

model_1_4 = Net_1_4()
optimizer = Adam(learning_rate=learning_rate)
loss_fn = BinaryCrossentropy()
model_1_4.compile(optimizer=optimizer, loss=loss_fn, metrics=['binary_accuracy'])

fit_log_1_4 = model_1_4.fit(loader_tr.load(), steps_per_epoch=loader_tr.steps_per_epoch, epochs=epochs, validation_data=loader_va.load(), validation_steps=loader_va.steps_per_epoch)

c:\Users\nick_\.virtualenvs\dotaprediction-eZ2WDOkz\lib\site-packages\spektral\data\utils.py:221: UserWarning: you are shuffling a 'DotaV1' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(a)


Epoch 1/50
25391/25391 [==============================] - 201s 8ms/step - loss: 0.6952 - binary_accuracy: 0.5122 - val_loss: 0.6946 - val_binary_accuracy: 0.5052
Epoch 2/50
25391/25391 [==============================] - 176s 7ms/step - loss: 0.6926 - binary_accuracy: 0.5167 - val_loss: 0.6924 - val_binary_accuracy: 0.5153
Epoch 3/50
25391/25391 [==============================] - 168s 7ms/step - loss: 0.6919 - binary_accuracy: 0.5208 - val_loss: 0.6925 - val_binary_accuracy: 0.5167
Epoch 4/50
25391/25391 [==============================] - 180s 7ms/step - loss: 0.6915 - binary_accuracy: 0.5232 - val_loss: 0.6922 - val_binary_accuracy: 0.5195
Epoch 5/50
25391/25391 [==============================] - 231s 9ms/step - loss: 0.6914 - binary_accuracy: 0.5238 - val_loss: 0.6912 - val_binary_accuracy: 0.5254
Epoch 6/50
25391/25391 [==============================] - 210s 8ms/step - loss: 0.6913 - binary_accuracy: 0.5249 - val_loss: 0.6913 - val_binary_accuracy: 0.5251
Epoch 7/50
25391/25391 [====

In [11]:
import plotly.graph_objects as go

epochs = list(range(1,len(fit_log_1_4.history['binary_accuracy'])+1))
training_accuracy = fit_log_1_4.history['binary_accuracy']
validation_accuracy = fit_log_1_4.history['val_binary_accuracy']

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=epochs,
    y=training_accuracy,
    mode='lines',
    name='Training'))
fig.add_trace(go.Scatter(
    x=epochs,
    y=validation_accuracy,
    mode='lines',
    name='Validation'))
fig.update_layout(
    template='none',
    xaxis_title='Epochs',
    yaxis_title="Accuracy")
fig.update_xaxes(tick0=0, dtick=1)
# fig.update_yaxes(range=[0.5,1])
fig.show()
fig.write_image(f'../images/model_1_4_accuracy.png', scale=5)
pd.DataFrame({'epoch': epochs, 'training_accuracy':training_accuracy, 'validation_accuracy':validation_accuracy}).to_csv('../images/model_1_4_accuracy.csv', index=False)

In [12]:
filehandler = open(f'../models/fit_log_1_4.pkl','wb')
pickle.dump(fit_log_1_4, filehandler)
filehandler = open(f'../models/model_1_4.pkl','wb')
pickle.dump(model_1_4, filehandler)

INFO:tensorflow:Assets written to: ram://7b05f3e4-e107-475b-a0a1-0b31bdb9debb/assets


c:\Users\nick_\.virtualenvs\dotaprediction-eZ2WDOkz\lib\site-packages\keras\saving\saved_model\layer_serialization.py:112: CustomMaskWarning:

Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.



INFO:tensorflow:Assets written to: ram://7d89233f-612d-4f45-b1d6-15caf5b93542/assets


c:\Users\nick_\.virtualenvs\dotaprediction-eZ2WDOkz\lib\site-packages\keras\saving\saved_model\layer_serialization.py:112: CustomMaskWarning:

Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.



### Model 1.5 - Attention Pooling

In [13]:
# Configuration
learning_rate = 0.001  # Learning rate
epochs = 50  # Number of training epochs
es_patience = 10  # Patience for early stopping
batch_size = 256  # Batch size

# Data loaders
loader_tr = BatchLoader(data_tr, batch_size=batch_size, epochs=epochs)
loader_va = BatchLoader(data_va, batch_size=batch_size)
loader_te = BatchLoader(data_te, batch_size=batch_size)

# Build model
class Net_1_5(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(19, activation='relu')
        self.pool1 = GlobalAttentionPool(19)
        self.dense = Dense(d.n_labels, activation="sigmoid")

    def call(self, inputs):
        x, a = inputs
        # x.shape is [n_batches, 5, n_ features]
        # a.shape is [n_batches, 5, 5]

        x = self.conv1([x, a])
        # x.shape is [n_batches, 5, channels] where I have set channels = features (20)

        x = self.pool1(x)
        # x.shape is [n_batches, channels]

        x = self.dense(x)
        # output.shape is [n_batches, 1]
        
        return x

model_1_5 = Net_1_5()
optimizer = Adam(learning_rate=learning_rate)
loss_fn = BinaryCrossentropy()
model_1_5.compile(optimizer=optimizer, loss=loss_fn, metrics=['binary_accuracy'])

fit_log_1_5 = model_1_5.fit(loader_tr.load(), steps_per_epoch=loader_tr.steps_per_epoch, epochs=epochs, validation_data=loader_va.load(), validation_steps=loader_va.steps_per_epoch)

c:\Users\nick_\.virtualenvs\dotaprediction-eZ2WDOkz\lib\site-packages\spektral\data\utils.py:221: UserWarning:

you are shuffling a 'DotaV1' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.



Epoch 1/50
25391/25391 [==============================] - 228s 9ms/step - loss: 0.6927 - binary_accuracy: 0.5159 - val_loss: 0.6917 - val_binary_accuracy: 0.5218
Epoch 2/50
25391/25391 [==============================] - 199s 8ms/step - loss: 0.6916 - binary_accuracy: 0.5219 - val_loss: 0.6913 - val_binary_accuracy: 0.5239
Epoch 3/50
25391/25391 [==============================] - 199s 8ms/step - loss: 0.6914 - binary_accuracy: 0.5239 - val_loss: 0.6912 - val_binary_accuracy: 0.5252
Epoch 4/50
25391/25391 [==============================] - 207s 8ms/step - loss: 0.6912 - binary_accuracy: 0.5246 - val_loss: 0.6913 - val_binary_accuracy: 0.5243
Epoch 5/50
25391/25391 [==============================] - 232s 9ms/step - loss: 0.6912 - binary_accuracy: 0.5251 - val_loss: 0.6914 - val_binary_accuracy: 0.5232
Epoch 6/50
25391/25391 [==============================] - 230s 9ms/step - loss: 0.6911 - binary_accuracy: 0.5254 - val_loss: 0.6914 - val_binary_accuracy: 0.5234
Epoch 7/50
25391/25391 [====

In [14]:
import plotly.graph_objects as go

epochs = list(range(1,len(fit_log_1_5.history['binary_accuracy'])+1))
training_accuracy = fit_log_1_5.history['binary_accuracy']
validation_accuracy = fit_log_1_5.history['val_binary_accuracy']

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=epochs,
    y=training_accuracy,
    mode='lines',
    name='Training'))
fig.add_trace(go.Scatter(
    x=epochs,
    y=validation_accuracy,
    mode='lines',
    name='Validation'))
fig.update_layout(
    template='none',
    xaxis_title='Epochs',
    yaxis_title="Accuracy")
fig.update_xaxes(tick0=0, dtick=1)
# fig.update_yaxes(range=[0.5,1])
fig.show()
fig.write_image(f'../images/model_1_5_accuracy.png', scale=5)
pd.DataFrame({'epoch': epochs, 'training_accuracy':training_accuracy, 'validation_accuracy':validation_accuracy}).to_csv('../images/model_1_5_accuracy.csv', index=False)

In [15]:
filehandler = open(f'../models/fit_log_1_5.pkl','wb')
pickle.dump(fit_log_1_5, filehandler)
filehandler = open(f'../models/model_1_5.pkl','wb')
pickle.dump(model_1_5, filehandler)

INFO:tensorflow:Assets written to: ram://ee72e0e3-707c-4145-8268-8824588a1440/assets


INFO:tensorflow:Assets written to: ram://ee72e0e3-707c-4145-8268-8824588a1440/assets


INFO:tensorflow:Assets written to: ram://ad7d20c4-5181-4e3a-abca-786392bddfb6/assets


INFO:tensorflow:Assets written to: ram://ad7d20c4-5181-4e3a-abca-786392bddfb6/assets


### Model 1.6 GATConv + Average Pooling

In [16]:
# Configuration
learning_rate = 0.001  # Learning rate
epochs = 50  # Number of training epochs
es_patience = 10  # Patience for early stopping
batch_size = 256  # Batch size

# Data loaders
loader_tr = BatchLoader(data_tr, batch_size=batch_size, epochs=epochs)
loader_va = BatchLoader(data_va, batch_size=batch_size)
loader_te = BatchLoader(data_te, batch_size=batch_size)

# Build model
class Net_1_6(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = GATConv(19, activation='relu')
        self.pool1 = GlobalAvgPool()
        self.dense = Dense(d.n_labels, activation="sigmoid")

    def call(self, inputs):
        x, a = inputs
        # x.shape is [n_batches, 5, n_ features]
        # a.shape is [n_batches, 5, 5]

        x = self.conv1([x, a])
        # x.shape is [n_batches, 5, channels] where I have set channels = features (20)

        x = self.pool1(x)
        # x.shape is [n_batches, channels]

        x = self.dense(x)
        # output.shape is [n_batches, 1]
        
        return x

model_1_6 = Net_1_6()
optimizer = Adam(learning_rate=learning_rate)
loss_fn = BinaryCrossentropy()
model_1_6.compile(optimizer=optimizer, loss=loss_fn, metrics=['binary_accuracy'])

fit_log_1_6 = model_1_6.fit(loader_tr.load(), steps_per_epoch=loader_tr.steps_per_epoch, epochs=epochs, validation_data=loader_va.load(), validation_steps=loader_va.steps_per_epoch)

c:\Users\nick_\.virtualenvs\dotaprediction-eZ2WDOkz\lib\site-packages\spektral\data\utils.py:221: UserWarning:

you are shuffling a 'DotaV1' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.



Epoch 1/50
25391/25391 [==============================] - 427s 17ms/step - loss: 0.6913 - binary_accuracy: 0.5262 - val_loss: 0.6908 - val_binary_accuracy: 0.5283
Epoch 2/50
25391/25391 [==============================] - 296s 12ms/step - loss: 0.6903 - binary_accuracy: 0.5324 - val_loss: 0.6901 - val_binary_accuracy: 0.5324
Epoch 3/50
25391/25391 [==============================] - 287s 11ms/step - loss: 0.6900 - binary_accuracy: 0.5336 - val_loss: 0.6901 - val_binary_accuracy: 0.5325
Epoch 4/50
25391/25391 [==============================] - 279s 11ms/step - loss: 0.6899 - binary_accuracy: 0.5337 - val_loss: 0.6899 - val_binary_accuracy: 0.5341
Epoch 5/50
25391/25391 [==============================] - 281s 11ms/step - loss: 0.6898 - binary_accuracy: 0.5342 - val_loss: 0.6900 - val_binary_accuracy: 0.5331
Epoch 6/50
25391/25391 [==============================] - 286s 11ms/step - loss: 0.6897 - binary_accuracy: 0.5348 - val_loss: 0.6903 - val_binary_accuracy: 0.5315
Epoch 7/50
25391/25391

In [19]:
import plotly.graph_objects as go

epochs = list(range(1,len(fit_log_1_6.history['binary_accuracy'])+1))
training_accuracy = fit_log_1_6.history['binary_accuracy']
validation_accuracy = fit_log_1_6.history['val_binary_accuracy']

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=epochs,
    y=training_accuracy,
    mode='lines',
    name='Training'))
fig.add_trace(go.Scatter(
    x=epochs,
    y=validation_accuracy,
    mode='lines',
    name='Validation'))
fig.update_layout(
    template='none',
    xaxis_title='Epochs',
    yaxis_title="Accuracy")
fig.update_xaxes(tick0=0, dtick=1)
# fig.update_yaxes(range=[0.5,1])
fig.show()
fig.write_image(f'../images/model_1_6_accuracy.png', scale=5)
pd.DataFrame({'epoch': epochs, 'training_accuracy':training_accuracy, 'validation_accuracy':validation_accuracy}).to_csv('../images/model_1_6_accuracy.csv', index=False)

In [18]:
filehandler = open(f'../models/fit_log_1_6.pkl','wb')
pickle.dump(fit_log_1_6, filehandler)
filehandler = open(f'../models/model_1_6.pkl','wb')
pickle.dump(model_1_6, filehandler)

INFO:tensorflow:Assets written to: ram://8742329e-711a-454c-b91d-20d06fba93f4/assets


INFO:tensorflow:Assets written to: ram://8742329e-711a-454c-b91d-20d06fba93f4/assets
c:\Users\nick_\.virtualenvs\dotaprediction-eZ2WDOkz\lib\site-packages\keras\saving\saved_model\layer_serialization.py:112: CustomMaskWarning:

Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.



INFO:tensorflow:Assets written to: ram://abbef555-889d-4fe7-9229-fe444bf287db/assets


INFO:tensorflow:Assets written to: ram://abbef555-889d-4fe7-9229-fe444bf287db/assets
c:\Users\nick_\.virtualenvs\dotaprediction-eZ2WDOkz\lib\site-packages\keras\saving\saved_model\layer_serialization.py:112: CustomMaskWarning:

Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.



### Model 1.7 - Feature selection

In [18]:
# Loop through each graph and scale feature matrix and drop following features: (col numbers are after removing attack_backswing (14) earlier)

# base_attack_time (12)
# attack_point (13)
# vision_day (14)
# vision_night (15)
# turn_rate (16)
# collision_size (17)

print('Selecting specific columns:')
for i in range(0,len(graphs_filt)): # match 0 has only 15 features, reason not known, skipping this
    if(i%100000==0):
        print(i)

    graphs_filt[i].x = graphs_filt[i].x[:,[0,1,2,3,4,5,6,7,8,9,10,11,18]]

Selecting specific columns:
0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000


In [19]:
# Train/valid/test split
d_fs = graphs_filt # Graph data

# np.random.seed(10)
idxs = np.random.permutation(len(d_fs))
split_va, split_te = int(0.64 * len(d_fs)), int(0.8 * len(d_fs)) #64% training, 16% validation, 20% test
# idx_tr, idx_va, idx_te = np.split(idxs, [split_va, split_te]) # use indices from earlier split
data_tr_fs = d_fs[idx_tr]
data_va_fs = d_fs[idx_va]
data_te_fs = d_fs[idx_te]

print(f'Training data: {np.round(len(data_tr_fs)/len(graphs_filt),2)*100}%')
print(f'Validation data: {np.round(len(data_va_fs)/len(graphs_filt),2)*100}%')
print(f'Test data: {np.round(len(data_te_fs)/len(graphs_filt),2)*100}%')

Training data: 64.0%
Validation data: 16.0%
Test data: 20.0%


In [20]:
# Configuration
learning_rate = 0.001  # Learning rate
epochs = 50  # Number of training epochs
es_patience = 10  # Patience for early stopping
batch_size = 256  # Batch size

# Data loaders
loader_tr = BatchLoader(data_tr_fs, batch_size=batch_size, epochs=epochs)
loader_va = BatchLoader(data_va_fs, batch_size=batch_size)
loader_te = BatchLoader(data_te_fs, batch_size=batch_size)

# Build model
class Net_1_7(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = GATConv(13, activation='relu')
        self.pool1 = GlobalAvgPool()
        self.dense = Dense(d.n_labels, activation="sigmoid")

    def call(self, inputs):
        x, a = inputs
        # x.shape is [n_batches, 5, n_ features]
        # a.shape is [n_batches, 5, 5]

        x = self.conv1([x, a])
        # x.shape is [n_batches, 5, channels] where I have set channels = features (20)

        x = self.pool1(x)
        # x.shape is [n_batches, channels]

        x = self.dense(x)
        # output.shape is [n_batches, 1]
        
        return x

model_1_7 = Net_1_7()
optimizer = Adam(learning_rate=learning_rate)
loss_fn = BinaryCrossentropy()
model_1_7.compile(optimizer=optimizer, loss=loss_fn, metrics=['binary_accuracy'])

fit_log_1_7 = model_1_7.fit(loader_tr.load(), steps_per_epoch=loader_tr.steps_per_epoch, epochs=epochs, validation_data=loader_va.load(), validation_steps=loader_va.steps_per_epoch)

c:\Users\nick_\miniconda3\envs\tf\lib\site-packages\spektral\data\utils.py:221: UserWarning: you are shuffling a 'DotaV1' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(a)


Epoch 1/50
25391/25391 [==============================] - 216s 8ms/step - loss: 0.6919 - binary_accuracy: 0.5202 - val_loss: 0.6920 - val_binary_accuracy: 0.5187
Epoch 2/50
25391/25391 [==============================] - 200s 8ms/step - loss: 0.6914 - binary_accuracy: 0.5255 - val_loss: 0.6919 - val_binary_accuracy: 0.5205
Epoch 3/50
25391/25391 [==============================] - 199s 8ms/step - loss: 0.6912 - binary_accuracy: 0.5264 - val_loss: 0.6919 - val_binary_accuracy: 0.5202
Epoch 4/50
25391/25391 [==============================] - 201s 8ms/step - loss: 0.6911 - binary_accuracy: 0.5275 - val_loss: 0.6917 - val_binary_accuracy: 0.5212
Epoch 5/50
25391/25391 [==============================] - 199s 8ms/step - loss: 0.6910 - binary_accuracy: 0.5280 - val_loss: 0.6916 - val_binary_accuracy: 0.5226
Epoch 6/50
25391/25391 [==============================] - 185s 7ms/step - loss: 0.6909 - binary_accuracy: 0.5286 - val_loss: 0.6917 - val_binary_accuracy: 0.5217
Epoch 7/50
25391/25391 [====

In [ ]:
import plotly.graph_objects as go

epochs = list(range(1,len(fit_log_1_7.history['binary_accuracy'])+1))
training_accuracy = fit_log_1_7.history['binary_accuracy']
validation_accuracy = fit_log_1_7.history['val_binary_accuracy']

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=epochs,
    y=training_accuracy,
    mode='lines',
    name='Training'))
fig.add_trace(go.Scatter(
    x=epochs,
    y=validation_accuracy,
    mode='lines',
    name='Validation'))
fig.update_layout(
    template='none',
    xaxis_title='Epochs',
    yaxis_title="Accuracy")
fig.update_xaxes(tick0=0, dtick=1)
# fig.update_yaxes(range=[0.5,1])
# fig.show()
fig.write_image(f'../images/model_1_7_accuracy.png', scale=5)
pd.DataFrame({'epoch': epochs, 'training_accuracy':training_accuracy, 'validation_accuracy':validation_accuracy}).to_csv('../images/model_1_7_accuracy.csv', index=False)

### Model 1.8 Hyperparameter Sweep

In [11]:
# Configuration
learning_rate = 0.001  # Learning rate
epochs = 40  # Number of training epochs
es_patience = 10  # Patience for early stopping
batch_size = 256  # Batch size

# Data loaders
loader_tr = BatchLoader(data_tr, batch_size=batch_size, epochs=epochs)
loader_va = BatchLoader(data_va, batch_size=batch_size)
loader_te = BatchLoader(data_te, batch_size=batch_size)

# Build model
class Net_1_8(Model):
    def __init__(self, channels):
        super().__init__()
        self.conv1 = GATConv(channels, activation='relu')
        self.pool1 = GlobalAvgPool()
        self.dense = Dense(d.n_labels, activation="sigmoid")

    def call(self, inputs):
        x, a = inputs
        # x.shape is [n_batches, 5, n_ features]
        # a.shape is [n_batches, 5, 5]

        x = self.conv1([x, a])
        # x.shape is [n_batches, 5, channels] where I have set channels = features (20)

        x = self.pool1(x)
        # x.shape is [n_batches, channels]

        x = self.dense(x)
        # output.shape is [n_batches, 1]
        
        return x

# Loop through range of num channels hyperparameter
channels = [5,10,15,19,25]
for i, channel in enumerate(channels):
    # Compile and train model
    model_1_8 = Net_1_8(channel)
    optimizer = Adam(learning_rate=learning_rate)
    loss_fn = BinaryCrossentropy()
    model_1_8.compile(optimizer=optimizer, loss=loss_fn, metrics=['binary_accuracy'])
    fit_log_1_8 = model_1_8.fit(loader_tr.load(), steps_per_epoch=loader_tr.steps_per_epoch, epochs=epochs, validation_data=loader_va.load(), validation_steps=loader_va.steps_per_epoch)

    # Save training and validation results
    epochs = list(range(1,len(fit_log_1_8.history['binary_accuracy'])+1))
    training_accuracy = fit_log_1_8.history['binary_accuracy']
    validation_accuracy = fit_log_1_8.history['val_binary_accuracy']
    pd.DataFrame({'epoch': epochs, 'training_accuracy':training_accuracy, 'validation_accuracy':validation_accuracy}).to_csv(f'../images/model_1_8_{i+1}_accuracy.csv', index=False)

    # Pickle model and training+validation log
    filehandler = open(f'../models/fit_log_1_8_{i}.pkl','wb')
    pickle.dump(fit_log_1_8, filehandler)
    filehandler = open(f'../models/model_1_8_{i}.pkl','wb')
    pickle.dump(model_1_8, filehandler)

c:\Users\nick_\miniconda3\envs\tf\lib\site-packages\spektral\data\utils.py:221: UserWarning: you are shuffling a 'DotaV1' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(a)


Epoch 1/40
25391/25391 [==============================] - 233s 9ms/step - loss: 0.6916 - binary_accuracy: 0.5241 - val_loss: 0.6907 - val_binary_accuracy: 0.5300
Epoch 2/40
25391/25391 [==============================] - 184s 7ms/step - loss: 0.6909 - binary_accuracy: 0.5292 - val_loss: 0.6905 - val_binary_accuracy: 0.5311
Epoch 3/40
25391/25391 [==============================] - 189s 7ms/step - loss: 0.6908 - binary_accuracy: 0.5298 - val_loss: 0.6908 - val_binary_accuracy: 0.5295
Epoch 4/40
25391/25391 [==============================] - 187s 7ms/step - loss: 0.6907 - binary_accuracy: 0.5303 - val_loss: 0.6903 - val_binary_accuracy: 0.5322
Epoch 5/40
25391/25391 [==============================] - ETA: 0s - loss: 0.6906 - binary_accuracy: 0.5310

### Model 1.9 MMR ranges

In [ ]:
# REMEMBER to choose optimal channels!

# Configuration
learning_rate = 0.001  # Learning rate
epochs = 50  # Number of training epochs
es_patience = 10  # Patience for early stopping
batch_size = 256  # Batch size

# Build model
class Net_1_9(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = GATConv(19, activation='relu')
        self.pool1 = GlobalAvgPool()
        self.dense = Dense(d.n_labels, activation="sigmoid")

    def call(self, inputs):
        x, a = inputs
        # x.shape is [n_batches, 5, n_ features]
        # a.shape is [n_batches, 5, 5]

        x = self.conv1([x, a])
        # x.shape is [n_batches, 5, channels] where I have set channels = features (20)

        x = self.pool1(x)
        # x.shape is [n_batches, channels]

        x = self.dense(x)
        # output.shape is [n_batches, 1]
        
        return x

mmrs = [1,2,3,4,5,6]

for group in mmrs:
    # Filter data for current mmr group
    filt = df_filters['filt_std'].values & df_filters[f'filt_mmr_{group}'].values
    step = 50000
    filt_vals = []
    for i in range(0,int(np.ceil(len(filt)/step))):
        start = i*step
        end = start+step
        filt_vals = np.append(filt_vals, filt[start:end])
        filt_vals = np.append(filt_vals, filt[start:end])

    # Get indices of True values in filters
    filt_idx = [i for i, x in enumerate(filt_vals) if x]

    # Select subset of graphs
    graphs_filt_mmr = graphs[filt_idx]

    # Train/valid/test split
    d = graphs_filt_mmr
    np.random.seed(10)
    idxs = np.random.permutation(len(d))
    split_va, split_te = int(0.7 * len(d)), int(len(d)) #70% training, 30% validation
    idx_tr, idx_va, idx_te = np.split(idxs, [split_va, split_te])
    data_tr = d[idx_tr]
    data_va = d[idx_va]

    # Data loaders
    loader_tr = BatchLoader(data_tr, batch_size=batch_size, epochs=epochs)
    loader_va = BatchLoader(data_va, batch_size=batch_size)

    # Compile and train model
    model_1_9 = Net_1_9()
    optimizer = Adam(learning_rate=learning_rate)
    loss_fn = BinaryCrossentropy()
    model_1_9.compile(optimizer=optimizer, loss=loss_fn, metrics=['binary_accuracy'])
    fit_log_1_9 = model_1_9.fit(loader_tr.load(), steps_per_epoch=loader_tr.steps_per_epoch, epochs=epochs, validation_data=loader_va.load(), validation_steps=loader_va.steps_per_epoch)

    # Save training and validation results
    epochs = list(range(1,len(fit_log_1_9.history['binary_accuracy'])+1))
    training_accuracy = fit_log_1_9.history['binary_accuracy']
    validation_accuracy = fit_log_1_9.history['val_binary_accuracy']
    pd.DataFrame({'epoch': epochs, 'training_accuracy':training_accuracy, 'validation_accuracy':validation_accuracy}).to_csv(f'../images/model_1_9_{group}_accuracy.csv', index=False)

    # Pickle model and training+validation log
    filehandler = open(f'../models/fit_log_1_9_{i}.pkl','wb')
    pickle.dump(fit_log_1_9, filehandler)
    filehandler = open(f'../models/model_1_9_{i}.pkl','wb')
    pickle.dump(model_1_9, filehandler)

### Extra useful code

In [117]:
loss = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)
print(f"Test accuracy: {np.round(loss[1],4)*100}%")

32/32 [==============================] - 0s 1ms/step - loss: nan - binary_accuracy: 0.5090
Test accuracy: 50.9%


c:\Users\nick_\.virtualenvs\dotaprediction-eZ2WDOkz\lib\site-packages\spektral\data\utils.py:221: UserWarning: you are shuffling a 'DotaV1' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(a)


In [141]:
loader_single = BatchLoader(data_va, batch_size=128)
loss = model.evaluate(loader_single.load(), steps=loader_single.steps_per_epoch)
print(f"Validation accuracy: {np.round(loss[1],4)*100}%")

50/50 [==============================] - 0s 1ms/step - loss: nan - binary_accuracy: 0.5800
Test accuracy: 57.99999999999999%


c:\Users\nick_\.virtualenvs\dotaprediction-eZ2WDOkz\lib\site-packages\spektral\data\utils.py:221: UserWarning: you are shuffling a 'DotaV1' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(a)
